# Demo: Export datacubes and files to EUDAT B2DROP

In this demo we will run some data analytics, create some plots and export the dataset and plot produced during these stages on our B2DROP folder.

**Pre-requisite 1:** a B2DROP account is required. You can login to B2DROP through B2ACCESS.

**Pre-requisite 2:** a NetCDF file in your B2DROP space (base folder).

**Pre-requisite 3:** a specific *Application Password* to be used for the file sharing through Ophidia, in order to avoid the sharing of the actual password with external applications.

<img src="imgs/b2drop_setup.png" alt="B2DROP Setup" width="1000">


After clicking on the *Create new app password* button a new password will be create. **The credentials will be shown just once so we need to store them before clicking on the *complete* button.**

Create a new **.netrc** file in the home directory with the credentials just generated (like in the example). This file will be used by Ophidia to authenticate to the B2DROP account and upload the files.

In [ ]:
%%bash
> $HOME/.netrc
cat <<'EOF' >> $HOME/.netrc
machine b2drop.eudat.eu
login  <username>
password <password>
EOF
chmod 0600 $HOME/.netrc

Import PyOphidia and connect to server instance

In [ ]:
from PyOphidia import cube, client
cube.Cube.setclient(read_env=True)

We can now download the NetCDF file from our B2DROP space, but before let's change the current folder used by Ophidia

In [ ]:
cube.Cube.fs(command='cd',
             dpath='/home/'+cube.Cube.client.username+'/', 
             display=True)

PyOphidia allows to download files from our B2DROP space by running the following class method (this is internally using the B2DROP APIs)

In [ ]:
cube.Cube.b2drop(src_path='tasmin_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc',action='get',dst_path='tasmin-b2drop.nc')

A new file named tasmin-b2drop.nc will be available in the home folder

In [ ]:
%%bash
ls -l $HOME/*.nc

Let's now import the dataset downloaded from B2DROP

In [ ]:
mycube = cube.Cube.importnc(src_path='tasmin-b2drop.nc', measure='tasmin', imp_dim='time', import_metadata='yes', 
                            imp_concept_level='d', ncores=2, hierarchy='oph_base|oph_base|oph_time', 
                            ioserver='ophidiaio_memory')

Compute the maximum value over the time series for each point in the spatial domain.

In [ ]:
mycube2 = mycube.reduce(operation='max',ncores=2)

Inspect the resulting cube

In [ ]:
mycube2.info()

Recall that the current folder is:

In [ ]:
cube.Cube.fs(command='ls', display=True)

Export the result into a nc file called tasmin-ophidia.nc

In [ ]:
mycube2.exportnc2(output_path='./',output_name='tasmin-ophidia')

A new file name tasmin-ophidia.nc will be available under the base folder /

In [ ]:
%%bash
ls -l $HOME/*.nc

We can reuse the PyOphidia class method also to upload data to the B2DROP space

In [ ]:
cube.Cube.b2drop(src_path='tasmin-ophidia.nc',action='put')

In a more integrated fashion, we can directly export a datacube into a NetCDF remotely stored on the B2DROP space.

In [ ]:
mycube2.to_b2drop()

Let's try to create a map with the datacube previously computed and store the file as map.png

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.util import add_cyclic_point
import numpy as np
import warnings
warnings.filterwarnings("ignore")

fig = plt.figure(figsize=(15, 6), dpi=100)

#Add Geo axes to the figure with the specified projection (PlateCarree)
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)

#Draw coastline and gridlines
ax.coastlines()

gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
gl.xlabels_top = False
gl.ylabels_right = False

data = mycube2.export_array()
lat = data['dimension'][0]['values'][ : ]
lon = data['dimension'][1]['values'][ : ]
var = data['measure'][0]['values'][ : ]
var = np.reshape(var, (len(lat), len(lon)))

#Wraparound points in longitude
var_cyclic, lon_cyclic = add_cyclic_point(var, coord=np.asarray(lon))
x, y = np.meshgrid(lon_cyclic,lat)

#Define color levels for color bar
levStep = (np.nanmax(var)-np.nanmin(var))/20
clevs = np.arange(np.nanmin(var),np.nanmax(var)+levStep,levStep)

#Set filled contour plot
cnplot = ax.contourf(x, y, var_cyclic, clevs, transform=projection,cmap=plt.cm.jet)
plt.colorbar(cnplot,ax=ax)

ax.set_aspect('auto', adjustable=None)

plt.title('Minimum Near-Surface Air Temperature (deg K)')
import os
plt.savefig(os.path.expanduser("~")+'/map.png',  bbox_inches='tight')
plt.show()

Using the PyOphidia method we can upload any type of file on B2DROP (we don't need to specify the *action* argument, since upload is the default action)

In [ ]:
cube.Cube.b2drop(src_path='map.png')

To clear your workspace before running other notebooks

In [ ]:
cube.Cube.deletecontainer(container='tasmin-b2drop.nc',force='yes')

In [ ]:
cube.Cube.list(level=2)